In [ ]:
# xml 2 txt -> yolo 형식으로 변환

# 경로를 지정해주자.

path = "./dataset"
annot_path = os.path.join(path,"annotations")
img_path = os.path.join(path,"images")
label_path = os.path.join(path,"labels")

import xml.etree.ElementTree as ET
import glob
import os
import json

# xml bbox 형식을 yolo bbox 형태로 변환하는 함수
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

classes = []

from tqdm import tqdm
files = glob.glob(os.path.join(annot_path, '*.xml'))

for f in tqdm(files): #
    basename = os.path.basename(f)
    filename = os.path.splitext(basename)[0]

    result = [] #

    tree = ET.parse(f)
    root = tree.getroot()
    # 너비 높이
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text  # label 이름

        if label not in classes:
            classes.append(label)

        index = classes.index(label)

        # bbox 위치 찾고 각각의 값들 리스트 안에 넣기.
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        # yolo 형식으로 바꾸기.
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # , 없애고 띄어쓰기 기준으로 분할
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        # class 리스트의 몇 번째에 속하는지
        result.append(f"{index} {bbox_string}") #


    if result: # 1개가 있다면 labels 폴더에 넣기!
        with open(os.path.join(label_path, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

---
xml 파일로 만들기

In [ ]:
<annotation>
	<folder>20211026 090012 H265-1</folder>
	<filename>20211026 090012 H265-1_2.jpg</filename>
	<path>C:\Users\itnj\Desktop\AI 데이터셋 프레임 파일\20211026 090012 H265-1\20211026 090012 H265-1_2.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>1920</width>
		<height>1080</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>person</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>1446</xmin>
			<ymin>217</ymin>
			<xmax>1603</xmax>
			<ymax>699</ymax>
		</bndbox>
	</object>
</annotation>

In [19]:
from xml.etree.ElementTree import Element, SubElement, ElementTree

filename = '20211026 090012 H265-1_2'
width = 1920
height = 1080
point1 = (1446, 1603)
point2 = (217, 699)
label = 'person'

root = Element('annotation')
SubElement(root, 'folder').text = '20211026 090012 H265-1'
SubElement(root, 'filename').text = filename + '.jpg'
SubElement(root, 'path').text = './contents/MyDrive/working/images' +  filename + '.jpg'
source = SubElement(root, 'source')
SubElement(source, 'database').text = 'Unknown'

size = SubElement(root, 'size')
SubElement(size, 'width').text = str(width)
SubElement(size, 'height').text = str(height)
SubElement(size, 'depth').text = '3'

SubElement(root, 'segmented').text = '0'

obj = SubElement(root, 'object')
SubElement(obj, 'name').text = label
SubElement(obj, 'pose').text = 'Unspecified'
SubElement(obj, 'truncated').text = '0'
SubElement(obj, 'difficult').text = '0'
bbox = SubElement(obj, 'bndbox')
SubElement(bbox, 'xmin').text = str(point1[0])
SubElement(bbox, 'ymin').text = str(point1[1])
SubElement(bbox, 'xmax').text = str(point2[0])
SubElement(bbox, 'ymax').text = str(point2[1])

tree = ElementTree(root)
tree.write('./' + filename +'.xml')

# 1개의 xml 파일 테스트

In [36]:
import os
import xml.etree.ElementTree as ET

xml_file ='20211026 090012 H265-1_2.xml'

# xml bbox 형식을 yolo bbox 형태로 변환하는 함수
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

classes = []


basename = os.path.basename(xml_file)
filename = os.path.splitext(basename)[0]

result = [] #

tree = ET.parse(xml_file)
root = tree.getroot()
# 너비 높이
width = int(root.find("size").find("width").text)
height = int(root.find("size").find("height").text)

for obj in root.findall('object'):
    label = obj.find("name").text  # label 이름

    if label not in classes:
        classes.append(label)

    index = classes.index(label)

    # bbox 위치 찾고 각각의 값들 리스트 안에 넣기.
    pil_bbox = [int(x.text) for x in obj.find("bndbox")]
    # yolo 형식으로 바꾸기.
    yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
    # , 없애고 띄어쓰기 기준으로 분할
    bbox_string = " ".join([str(x) for x in yolo_bbox])
    # class 리스트의 몇 번째에 속하는지
    result.append(f"{index} {bbox_string}") #
print(result)

['0 0.43307291666666664 1.0657407407407407 -0.6401041666666667 -0.837037037037037']


In [40]:
print(label)

person


In [37]:
print(width, height)

1920 1080


In [39]:
print(pil_bbox)
print(yolo_bbox)

[1446, 1603, 217, 699]
[0.43307291666666664, 1.0657407407407407, -0.6401041666666667, -0.837037037037037]
